In [4]:
import pyspark
from delta import *

In [5]:
builder=(pyspark.sql.SparkSession.builder.appName("Delete_Delta_Lake_Table")
        .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
        )

In [6]:
spark=configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/vk/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/vk/.ivy2/cache
The jars for the packages stored in: /Users/vk/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6ed8310b-be92-4876-8248-795aa80e70b1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 87ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

23/04/08 22:59:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
df = spark.createDataFrame([("QAZ",1),("WSX",2),("EDC",3),("RFV",4)]).toDF("Name","Age")

In [8]:
df.rdd.getNumPartitions()

8

In [15]:
df.write.format("delta").mode("overwrite").save("/tmp/deltatable1")

In [16]:
spark.read.format("delta").load("/tmp/deltatable1/").show()

+----+---+
|Name|Age|
+----+---+
| QAZ|  1|
| WSX|  2|
| EDC|  3|
| RFV|  4|
+----+---+



In [17]:
%cat "/tmp/deltatable1/_delta_log/00000000000000000000.json"

{"commitInfo":{"timestamp":1680966230813,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"5","numOutputRows":"4","numOutputBytes":"3213"},"engineInfo":"Apache-Spark/3.3.2 Delta-Lake/2.2.0","txnId":"cd4e7509-2de6-4fdc-9b53-6199fd72c5c0"}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"7f108376-f032-4ccf-ba41-a05c60365667","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"Name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Age\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1680966230511}}
{"add":{"path":"part-00001-75678ede-91ec-4361-b4df-9170ffd6a704-c000.snappy.parquet","partitionValues":{},"size":708,"modificationTime":1680966230720,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"Name

In [18]:
import pyarrow.parquet as pq


In [19]:
parquet_file = pq.ParquetFile("/tmp/deltatable1/part-00000-a7aaac08-9f9a-4d44-858b-581f2d98f3f0-c000.snappy.parquet")

In [20]:
parquet_file.schema

required group field_id=-1 spark_schema {
  optional binary field_id=-1 Name (String);
  optional int64 field_id=-1 Age;
}

In [21]:
df1=spark.createDataFrame([("RFV",5),("TGB",6)]).toDF("Name","Age")

In [24]:
df1.write.format("delta").mode("append").save("/tmp/deltatable1")

In [25]:
spark.read.format("delta").load("/tmp/deltatable1/").show()

+----+---+
|Name|Age|
+----+---+
| QAZ|  1|
| WSX|  2|
| TGB|  6|
| EDC|  3|
| RFV|  5|
| RFV|  4|
+----+---+



In [54]:
%cat "/tmp/deltatable1/_delta_log/00000000000000000001.json"

{"commitInfo":{"timestamp":1680966751885,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"3","numOutputRows":"2","numOutputBytes":"1797"},"engineInfo":"Apache-Spark/3.3.2 Delta-Lake/2.2.0","txnId":"0e6c15eb-ad67-469d-a53c-eb841e39b365"}}
{"add":{"path":"part-00003-189fec81-ca42-43cb-84ad-fb950110bf35-c000.snappy.parquet","partitionValues":{},"size":708,"modificationTime":1680966751864,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"Name\":\"RFV\",\"Age\":5},\"maxValues\":{\"Name\":\"RFV\",\"Age\":5},\"nullCount\":{\"Name\":0,\"Age\":0}}"}}
{"add":{"path":"part-00007-e6fcc47a-f486-440b-a6f4-431b13203703-c000.snappy.parquet","partitionValues":{},"size":708,"modificationTime":1680966751880,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"Name\":\"TGB\",\"Age\":6},\"maxValues\":{\"Name\":\"TGB\",\"Age\":6},\"nullCount\":{\"Name\":0,\"

# DELETE ROWS WHERE ARE GREATER THAN 5

In [55]:
import delta
import pyspark.sql.functions as f

In [56]:
dt = delta.DeltaTable.forPath(spark,"/tmp/deltatable1/")
dt.delete(f.column("Age") > 5)

In [57]:
spark.read.format("delta").load("/tmp/deltatable1/").show()

+----+---+
|Name|Age|
+----+---+
| QAZ|  1|
| WSX|  2|
| EDC|  3|
| RFV|  5|
| RFV|  4|
+----+---+



In [59]:
%ls -lrt "/tmp/deltatable1/_delta_log/"

total 24
-rw-r--r--@ 1 vk  wheel  2072 Apr  8 23:03 00000000000000000000.json
-rw-r--r--@ 1 vk  wheel  1013 Apr  8 23:12 00000000000000000001.json
-rw-r--r--@ 1 vk  wheel   684 Apr  8 23:28 00000000000000000002.json


In [60]:
%cat "/tmp/deltatable1/_delta_log/00000000000000000002.json"

{"commitInfo":{"timestamp":1680967735061,"operation":"DELETE","operationParameters":{"predicate":"[\"(Age > 5L)\"]"},"readVersion":1,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"0","numAddedChangeFiles":"0","executionTimeMs":"463","numDeletedRows":"1","scanTimeMs":"319","numAddedFiles":"0","rewriteTimeMs":"144"},"engineInfo":"Apache-Spark/3.3.2 Delta-Lake/2.2.0","txnId":"f8c8badb-6f0a-49b6-bb5f-75751a3e277b"}}
{"remove":{"path":"part-00007-e6fcc47a-f486-440b-a6f4-431b13203703-c000.snappy.parquet","deletionTimestamp":1680967735052,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":708}}
